In [3]:
import pandas as pd, json, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
cluster=json.loads(file('../json/cluster.json','r').read())
citysave=json.loads(file('../json/citysave.json','r').read())

In [5]:
import wolframalpha
app_id='RV7XH4-KUYXP36YXX'
#csaladenes@yahoo.com
client = wolframalpha.Client(app_id)

In [6]:
unicities={}
for i in cluster:
    if cluster[i] not in unicities:
        unicities[cluster[i]]=citysave[i]['country']

In [7]:
G={}
error=[]

In [8]:
uk=unicities.keys()
for h in range(1800,2700):
    c=uk[h]
    if c not in G.keys()+error:
        print h,
        ys={"pop":0,"nearby":{}}
        q='population of '+c+', '+unicities[c]
        try:
            res = client.query(q)
            for i in range(len(res['pod'])):
                if res['pod'][i]['@title']=="Result":
                    x=res['pod'][i]['subpod']['plaintext']
                    popul=x[:x.find('people')-1]
                    if 'mill' in popul:
                            popul=float(popul[:popul.find('mill')-1])*1000000.0
                    ys['pop']=int(popul)
                if res['pod'][i]['@title']=="Nearby cities":
                    x=res['pod'][i]['subpod']['plaintext'].split('\n')
                    for y in x[:-1]:
                        people=y[y.rfind('|')+2:y.find('people')-1]
                        if 'mill' in people:
                            people=float(people[:people.find('mill')-1])*1000000.0
                        km=float(y[y.find('|')+2:y.find('km')-1])
                        ys['nearby'][y.split('|')[0].split(',')[0].strip()]={"km":km,"people":int(people)}
            G[c]=ys
        except: error.append(c)

1800 1801 1802 1803 1804 1805 1806 1807 1808 1809 1810 1811 1812 1813 1814 1815 1816 1817 1818 1819 1820 1821 1822 1823 1824 1825 1826 1827 1828 1829 1830 1831 1832 1833 1834 1835 1836 1837 1838 1839 1840 1841 1842 1843 1844 1845 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855 1856 1857 1858 1859 1860 1861 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 

In [11]:
file("../json/pop3.json",'w').write(json.dumps(G))
file("../json/pop3e.json",'w').write(json.dumps(error))

Postprocessing

In [ ]:
G=json.loads(file('../json/pop3.json','r').read())
error=json.loads(file('../json/pop3e.json','r').read())

In [12]:
error2=[]

In [27]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

In [28]:
for c in error:
    if c not in G.keys()+error2:
        ys={"pop":0,"nearby":{}}
        q=remove_accents(strip_accents('population of '+c.split('/')[0]+', '+unicities[c]))
        res = client.query(q)
        if 'pod' in res:
            for i in range(len(res['pod'])):
                if res['pod'][i]['@title']=="Result":
                    x=res['pod'][i]['subpod']['plaintext']
                    if 'available' not in x:
                        popul=x[:x.find('people')-1]
                        if 'mill' in popul:
                            popul=popul[:popul.find('mill')-1]
                            if '|' in popul:popul=popul.split('|')[1].strip()
                        ys['pop']=int(float(popul)*1000000.0)
                if res['pod'][i]['@title']=="Nearby cities":
                    x=res['pod'][i]['subpod']['plaintext'].split('\n')
                    if 'available' not in x:
                        for y in x[:-1]:
                            people=y[y.rfind('|')+2:y.find('people')-1]
                            if 'mill' in people:
                                people=float(people[:people.find('mill')-1])*1000000.0
                            km=float(y[y.find('|')+2:y.find(' km ')])
                            ys['nearby'][y.split('|')[0].split(',')[0].strip()]={"km":km,"people":int(people)}
                        G[c]=ys
                        print 'success',c
        else: 
            print 'error',c
            error2.append(c)

success Jonkoping
success Balikpapan
error Port Hope Simpson
success Brainerd
success Boston
error Arviat
success Kokhanok
success Lodwar
success La Rioja
success Manado
error Kimmirut
success Aarhus
success Pullman
success Mendoza
error Kuujjuaq
success Puerto Cabezas
success Kuqa
success Mazatlan
success Surigao
error Wanzhou
success Igarka
success Johor Bahru
success Puerto Vallarta
error Snare Lake
success Albury
error Deadmans Cay
success Selma
error Kingscote
success Al Hoceima
success Inhambane
success Great Falls
success Moncton
success Aurangabad
success Faisalabad
error Sveg
success Petrolina
success Golmud
success Meixian
success Nagasaki
success Maraba
success Grand Junction
success Haikou
success Ottawa
success Kahului
success Pituffik
error Kaimana-Papua Island
error Carajas
error Déline
success Fuyuan
success Valenca
success Port-au-Prince
success Vero Beach
success Madurai
success Mosjoen
success Hubli
error Makkovik
success Myeik
error Kosrae
error Amami
success New Or

In [29]:
file("../json/pop3b.json",'w').write(json.dumps(G))
file("../json/pop3eb.json",'w').write(json.dumps(error2))
print len(G),len(error),len(error2)

746 531 88


Re-reparse

In [ ]:
G=json.loads(file('../json/pop3b.json','r').read())
error=json.loads(file('../json/pop3eb.json','r').read())

In [31]:
error3=[]
error4=[]
uk=unicities.keys()
for h in range(1800,2700):
    c=uk[h]
    if c not in G:
        error3.append(c)
print len(error3)

154


In [33]:
for c in error3:
    if c not in G.keys()+error4:
        ys={"pop":0,"nearby":{}}
        q=remove_accents(strip_accents('population of '+c.split('/')[0]+', '+unicities[c]))
        res = client.query(q)
        good=True
        if 'pod' in res:
            for i in range(len(res['pod'])):
                try:
                    if res['pod'][i]['@title']=="Result":
                        x=res['pod'][i]['subpod']['plaintext']
                        if 'available' not in x:
                            popul=x[:x.find('people')-1]
                            if 'mill' in popul:
                                popul=popul[:popul.find('mill')-1]
                                if '|' in popul:popul=popul.split('|')[1].strip()
                            ys['pop']=int(float(popul)*1000000.0)
                    if res['pod'][i]['@title']=="Nearby cities":
                        x=res['pod'][i]['subpod']['plaintext'].split('\n')
                        if 'available' not in x:
                            for y in x[:-1]:
                                people=y[y.rfind('|')+2:y.find('people')-1]
                                if 'mill' in people:
                                    people=float(people[:people.find('mill')-1])*1000000.0
                                km=float(y[y.find('|')+2:y.find(' km ')])
                                ys['nearby'][y.split('|')[0].split(',')[0].strip()]={"km":km,"people":int(people)}
                            G[c]=ys
                            print 'success',c
                            good=False
                except: pass
        if good: 
            print 'error',c
            error4.append(c)

error Leonora
error Koh Samui
error Isla De Culebra
error Chlef
error La Grande Riviere
error Sogndal
error Kitaakita
error La Palma
error Texada
error Philippines
error Daqing
error Aden
error Pico
error Dominica
error Gode
error Hassi Messaoud
error Naryan-Mar
error Kelle
error Essendon
error Adak Island
error Narrandera
error Ghat
error San Andres Island
error Solomon Islands
error Alor Island
error Leros
error Moro
error Rost
error Kinmen
error Tongatapu
error Værøy
error Male
error Morón
error Tachilek
error Algeria
error Panjgur
error Mulu
error Lijiang
error Goa
error Ha'il
error Arar
error North Eleuthera
error Germany
error Kowanyama
error Taraz
error Vilhelmina
error Barre-Montpelier
error Providenciales
error Papa Westray
error Lakeba Island
error Nosy Be
error Gaua Island
error Ravensthorpe
error Karakelong Island
error Paro
error Atiu Island
error Olga Bay
error Sao Jorge
error Sakon Nakhon
error Thandwe
error Sorriso
error São Tomé and Principe
error Kiritimati
error Lenc

In [34]:
file("../json/pop3c.json",'w').write(json.dumps(G))
file("../json/pop3ec.json",'w').write(json.dumps(error4))
print len(G),len(error4)

746 154
